<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2020-1/Insumos/BonoCupVarBondeD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 72, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 72 (delta 23), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (72/72), done.


In [2]:
%%R
library(dplyr)
install.packages("data.table")
require(data.table)
#FUNCIÓN DE INTERPOLACIÓN ALAMBRADA

talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}


/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: trying URL 'https://cran.rstudio.com/src/contrib/data.table_1.12.2.tar.gz'

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/

**Características**

Valor nominal: Generalmente, 100 pesos pero varía por emisora.

Plazo: de 1 a 360 días, según las necesidades de financiamiento de la empresa emisora.

Rendimiento: al igual que los bonos, este instrumento se compra a descuento respecto de su valor nominal, pero por lo general pagan una sobretasa referenciada a tasa fija o variable.

Garantía: este título, por ser un pagaré, no ofrece ninguna garantía, por lo que es importante evaluar bien al emisor. 



In [0]:
%%R
btasadesc="RiesgosFinancieros/2020-1/Insumos/tasa_guber.txt"
btasadescst="RiesgosFinancieros/2020-1/Insumos/tasa_guber_st.txt"
btasafondeo="RiesgosFinancieros/2020-1/Insumos/tfondeo.txt"
fval=as.Date("20180629",format="%Y%m%d") #Fecha de valoración
plazos=cbind( 358,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon=cbind( 28,	28, 28, 28, 28) #plazos fijos de cada cupón
contratos=cbind(22000, -29000, 29000, -46000, 10000) #posición invertida
nominal=100
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada


In [0]:
%%R
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
X1_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos=data.frame(data1[1,2:m_orig])
n=n-1

data3<-read.table(btasadescst)
n3<-nrow(data3)
m3_orig=ncol(data3)
X3_orig=data.frame(data3[2:n3,2:m3_orig])
X3a_orig=mutate(data.frame(data1[2:n,1:m_orig]), V1=as.Date(V1,format="%Y%m%d"))
nodos3=data.frame(data3[1,2:m3_orig])
n3=n3-1


data2<-read.table(btasafondeo)
n2<-nrow(data2)
X2_orig=data.frame(data2[2:n2,1:2])
X2a_orig=mutate(X2_orig, V1=as.Date(V1,format="%Y%m%d"), V2=as.numeric(as.character(V2)))
tfh=seq(as.Date("20170629",format="%Y%m%d"), as.Date("20180629",format="%Y%m%d"), "days") #sucesión de dias para tasa fondeo
#tfhd=data.frame(ID=1:count(tfh),fecha=tfh)
tfhd=CJ(fecha=tfh)
#tfhd

In [0]:

%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
tfhd=setDT(tfhd)[, join_date := tfh][order(-join_date)]
X2a_orig=setDT(X2a_orig)[, join_date := V1][order(-join_date)]
X2a_orig=X2a_orig[tfhd, on = .(join_date), roll = Inf] 


In [14]:
%%R
#Cruzar la sucesión de todos los días versus el de tasa de fondeo
#buscar fecha de valuación en tfondeo
tf_act=X2a_orig[fecha==fval,]$V2/100
tf_int=X2a_orig[fecha<=fval & fecha>=(fval-plazocupon[1])]$V2/100

X1_orig=setDT(X1_orig)[, join_date := V1][order(-join_date)] #Para alinear con valor presente y tasa de fondeo.
X2_pr=X2a_orig[X1_orig, on = .(join_date)] #Se alinea la tasa de fondeo por fecha con la tasa de valor presente.
X2_pr

             V1   V2  join_date      fecha       i.V1       i.V2         V3
  1: 2018-06-29 7.95 2018-06-29 2018-06-29 2018-06-29 0.07906000 0.07808887
  2: 2018-06-28 7.81 2018-06-28 2018-06-28 2018-06-28 0.07722139 0.07808887
  3: 2018-06-27 7.79 2018-06-27 2018-06-27 2018-06-27 0.07722139 0.07808887
  4: 2018-06-26 7.76 2018-06-26 2018-06-26 2018-06-26 0.07813334 0.07808887
  5: 2018-06-25 7.74 2018-06-25 2018-06-25 2018-06-25 0.07722139 0.07808887
 ---                                                                       
251: 2017-07-14 7.06 2017-07-14 2017-07-14 2017-07-14 0.07906000 0.07579952
252: 2017-07-13 7.09 2017-07-13 2017-07-13 2017-07-13 0.07881962 0.07705368
253: 2017-07-12 7.03 2017-07-12 2017-07-12 2017-07-12 0.07906000 0.07736246
254: 2017-07-11 7.06 2017-07-11 2017-07-11 2017-07-11 0.07906000 0.07625908
255: 2017-07-10 6.90 2017-07-10 2017-07-10 2017-07-10 0.07906000 0.07671229
             V4         V5         V6         V7         V8         V9
  1: 0.07709264 0

In [16]:
%%R      
m=ncol(plazos)


N=as.integer(plazos/plazocupon)+1 #número de cupones a pagar
VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
ulNomT=matrix(0,1,sum(N)) #vector de contratos a final de flujo


plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales
ddv=plazocupon-plazini #dias trasncurridos del cupón vigente
tfcupon=matrix(0,1,m)  #El primero cupón de cada bono
tfcupondev=matrix(0,1,m) #cupón de los días devengados
tfcupgen=((1+tf_act/360)^(plazocupon[1])-1)*360/plazocupon[1] #el segundo al último cupón de todos los bonos
#calcula cupones de bonos
for (j in (1:m))
{
  tfcupondev[j]=(prod(1+tf_int[(1:ddv[j])]/360)-1)*360/ddv[j]
  tfcupon[j]=((1+tfcupondev[j]*ddv[j]/360)*(1+tf_act/360)^(plazocupon[1]-ddv[j])-1)*360/plazocupon[1]
}


for (j in (1:m))
{
  if (j==1)
  {
    VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
    plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    ulNomT[,sum(N[1:j])]=contratos[j]
    tasafijaT[,1]=tfcupon[j]
    tasafijaT[,2:sum(N[1:j])]=seq(tfcupgen,tfcupgen)
  }
  else
  {
    VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
    contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])					
    plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
    tasafijaT[,(sum(N[1:j-1])+1)]=tfcupon[j]
    tasafijaT[,(sum(N[1:j-1])+2):sum(N[1:j])]=seq(tfcupgen,tfcupgen)
    ulNomT[,sum(N[1:j])]=contratos[j]
  }
}

Xvp=matrix(0,n,ncol(VTplazos))
Xst=matrix(0,n,ncol(VTplazos))

for (i in (1:n))
{
  Xvp[i,]=if(itpl==0){approx(nodos,X_orig[i,],VTplazos)$y}else{talamb(nodos,X_orig[i,],VTplazos)}
  Xst[i,]=if(itpl==0){approx(nodos3,X3_orig[i,],VTplazos)$y}else{talamb(nodos3,X3_orig[i,],VTplazos)}
  
}

head(Xvp)
head(Xst)

           [,1]       [,2]       [,3]       [,4]       [,5]       [,6]
[1,] 0.02413746 0.02435114 0.02463158 0.02487811 0.02509921 0.02532031
[2,] 0.02413746 0.02433222 0.02458617 0.02484177 0.02509861 0.02535546
[3,] 0.02420988 0.02442517 0.02465379 0.02485744 0.02504237 0.02522729
[4,] 0.02420988 0.02442517 0.02465379 0.02485744 0.02504237 0.02522729
[5,] 0.02420988 0.02448194 0.02479004 0.02504915 0.02527152 0.02549389
[6,] 0.02420248 0.02450086 0.02483545 0.02510616 0.02532896 0.02555175
           [,7]       [,8]       [,9]      [,10]      [,11]      [,12]
[1,] 0.02554670 0.02578262 0.02601853 0.02625112 0.02646373 0.02667634
[2,] 0.02559837 0.02581620 0.02603404 0.02625385 0.02648552 0.02671719
[3,] 0.02543667 0.02569005 0.02594343 0.02619638 0.02644676 0.02669715
[4,] 0.02543667 0.02569005 0.02594343 0.02619638 0.02644676 0.02669715
[5,] 0.02574089 0.02603221 0.02632354 0.02660435 0.02682211 0.02703987
[6,] 0.02580457 0.02611144 0.02641832 0.02671319 0.02693607 0.02715896
      


$$\Large V=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c_{p_i}}\cdot p_c/360}{(1+t_{vp_{p_i}} \cdot p_i/360)} + \frac{N\cdot C}{(1+t_{vp_{p_n}} \cdot p_n/360)}$$
	Donde: \\
$N$: Valor Nominal del bono\
$C$: Número de contratos\
$p_{c}$: Plazo fijo para cada pago de intereses del cupón.\
$p_{i}$: Plazo acumulado (en días) al cupón $i$.\
$t_{c_{p_i}}$: Tasa cupón variable, se obtiene de la curva subyacente que le corresponda, casi siempre con la tasa forward entre $p_{i-1}$ y $p_i$.\
$t_{vp_{p_i}}$: Tasa valor presente que depende de la curva de bonos según el plazo acumulado al pago del cupón $i$.


En general tenemos dos factores de riesgo subyacente (la curva de valor presente y la curva de cupones) pero como cada cupón tiene ``$n$'' flujos entonces tiene $n$ factores de riesgo para los valores presentes y $n$ factores de riesgo para los cupones, por lo que tienen $2n$ factores de riesgo específicos que provienen de dos factores de riesgo subyacentes.
Para el caso de $K$ bonos cupón variable el número de factores de riesgo sería $2\sum_{i=1}^{K}n_i$, donde $n_i$ es el número de cupones a pagar del bono $i$.

In [36]:
%%R                

bondeD=function(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, Xst, N)
{ 
  V0=matrix(0,1,nrow(as.matrix(N)))
  V0f=((((contratosT*(tasafijaT)*(plazocuponT/360))+ulNomT)/(1+(Xvp+Xst)*VTplazos/360)))*nominal
  for (j in (1:nrow(as.matrix(N))))
  {
    if(j==1)
    {
      V0[j]=sum(V0f[j:N[j]])
    }
    else
    {
      V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
    }
  }
  V0
} 

V0=bondeD(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp[1,], Xst[1,], N)

V0

        [,1]     [,2]    [,3]     [,4]     [,5]
[1,] 2155549 -2840104 2817830 -4342999 933071.2
